In [1]:
from youtube_constants import YOUTUBE_API_ACCESS_TOKEN, CHANNEL_ID

In [2]:
print(CHANNEL_ID)

UCphQA4DKYX14O32KDdf_mSQ


In [3]:
from youtube_data import get_all_video_in_channel, get_videos_from_list, chunks

In [4]:

respo, video_links = get_all_video_in_channel(api_key=YOUTUBE_API_ACCESS_TOKEN,
                                               channel_id=CHANNEL_ID)


In [5]:
# formatting the data outputs fo the video links for the next api call
import pandas as pd



dfvideolinks = pd.DataFrame(video_links)


dftemp = dfvideolinks.drop(columns=["id", "kind"]).merge(dfvideolinks["id"].apply(pd.Series), right_index=True, left_index=True)
dftemp_final_video_channel_list = dftemp.drop(columns="snippet").merge(dftemp["snippet"].apply(pd.Series), right_index=True, left_index=True)
dftemp_final_video_channel_list["thumbnail"] = dftemp_final_video_channel_list["thumbnails"].map(lambda x: x.get("high", {}).get("url", None))
# final column pass for part 1 
dftemp_final_video_channel_list = dftemp_final_video_channel_list.drop(columns=["thumbnails"])


In [6]:
# showing what the df looks like so far
dftemp_final_video_channel_list.sample(2)

,etag,videoLink,kind,videoId,publishedAt,channelId,title,description,channelTitle,liveBroadcastContent,publishTime,thumbnail
42,dN1CC3rLme-VAOL2AE367X_jYJA,https://www.youtube.com/watch?v=nwI9STTjCbI,youtube#video,nwI9STTjCbI,2022-09-02T20:39:35Z,UCphQA4DKYX14O32KDdf_mSQ,"Special Council Meeting August 24, 2022 (1)",,Town of Fort Fairfield,none,2022-09-02T20:39:35Z,https://i.ytimg.com/vi/nwI9STTjCbI/hqdefault.jpg
54,xwWNVmQPbCZWAO70CHkjWYMmSho,https://www.youtube.com/watch?v=foX46E_SxQI,youtube#video,foX46E_SxQI,2022-06-08T11:46:59Z,UCphQA4DKYX14O32KDdf_mSQ,"Budget Advisory Committee June 7, 2022",,Town of Fort Fairfield,none,2022-06-08T11:46:59Z,https://i.ytimg.com/vi/foX46E_SxQI/hqdefault.jpg


In [7]:
print(f"videos in channel: {dftemp_final_video_channel_list.shape[0]}")

videos in channel: 66


In [8]:
# now passing various video ids to get more information about the video


list_of_video_lists = list(chunks(l=dftemp_final_video_channel_list["videoId"].tolist(), n=5))

In [9]:
list_resps = []
list_videos_dfs = []

for video_list in list_of_video_lists:
    # now we have a lists of lists that we can call the api since we cant pass in 50+ videos at one time
    respov, items_ = get_videos_from_list(api_key=YOUTUBE_API_ACCESS_TOKEN, video_id_list=video_list)
    list_resps.append(respov)
    list_videos_dfs.append(pd.DataFrame(items_))


In [10]:
big_df_videos = pd.concat(list_videos_dfs).reset_index(drop=True)
print(big_df_videos.shape[0])

66


In [11]:
DF_VIDEOS = dftemp_final_video_channel_list.merge(big_df_videos, right_on=["videoId"], left_on=["videoId"])

In [12]:
# added more data to  the DF VIDEOS
DF_VIDEOS.sample(2)


,etag,videoLink,kind,videoId,publishedAt,channelId,title,description,channelTitle,liveBroadcastContent,publishTime,thumbnail,duration_seconds,duration_approx_minutes,viewCount,likeCount,favoriteCount,commentCount
52,TQj9a87sxOlq4bcir8Rdz1wO-b4,https://www.youtube.com/watch?v=0Hepid8MmwE,youtube#video,0Hepid8MmwE,2022-06-10T12:31:58Z,UCphQA4DKYX14O32KDdf_mSQ,Budget Workshop #2 6.9.22 (3),,Town of Fort Fairfield,none,2022-06-10T12:31:58Z,https://i.ytimg.com/vi/0Hepid8MmwE/hqdefault.jpg,3601.0,60.0,43,0,0,0
1,5iAWO-lQ7JLl9RYiL9sqUsc2EsE,https://www.youtube.com/watch?v=ZUHU4d5AD9U,youtube#video,ZUHU4d5AD9U,2024-06-13T12:09:20Z,UCphQA4DKYX14O32KDdf_mSQ,(#1) 12June24 Regular Town Council Meeting,,Town of Fort Fairfield,none,2024-06-13T12:09:20Z,https://i.ytimg.com/vi/ZUHU4d5AD9U/hqdefault.jpg,3604.0,60.0,203,4,0,0


In [13]:
DF_VIDEOS.to_csv("../outputs/sample-test.csv", index=False)

In [14]:
# mnow to get caption text
from youtube_data import get_transcript_from_video_id



In [15]:
import jsonlines

l_transcripts = []
fout_path = "../outputs/videos-transcripts-sample.jsonl"

with jsonlines.open(fout_path, "w") as fout:
    
    for i, row in DF_VIDEOS.iterrows():

        new_i = {}
        video_id = row["videoId"]
        new_i["videoId"] = video_id
        try:
            new_i["captions_raw"] = get_transcript_from_video_id(row["videoId"])
            new_i["caption_text_flag"] = True
        except Exception as error:
            print(f"An error occurred for video {video_id} @ index {i}:\n", error)
            new_i["captions_raw"] = None
            new_i["caption_text_flag"] = False
            print(new_i)

        
        fout.write(new_i)
        l_transcripts.append(new_i)
        

An error occurred for video 4NTecDkUD6I @ index 41:
 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=4NTecDkUD6I! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem!
{'videoId': '4NTecDkUD6I', 'captions_raw': None, 'caption_text_flag': False}
An error occurred for video dRbZCyamcyg @ index 46:
 no element found: line 1, column 0
{'videoId': 'dRbZCyamcyg', 'captions_raw': None, 'caption_text_flag': False}
An error occurred for video XY-MOEmH11M @ index 50:
 no element found: line 1, column 0
{'videoId': 'XY-MOEmH11M', 'captions_raw': N

In [16]:
# uncomment if you want to use the file output
# add path to where your transcripts jsonl file is

# path_transcripts_jsonl = "../outputs/videos-transcripts-sample.jsonl"

# df_transcripts = pd.read_json(path_transcripts_jsonl, lines=True)


# or if you want to use the list
df_transcripts = pd.DataFrame(l_transcripts)

In [17]:

# making final merged df
merged_df = DF_VIDEOS.merge(df_transcripts, right_on="videoId", left_on="videoId", how="outer")




In [18]:
column_rename_map = {

  "channelId": 'channel_id',
  "channelTitle":  'channel_title',
  "videoId": 'vid_id', 
  "title": 'vid_title',
  "description": 'vid_desc',
  "duration_approx_minutes": 'vid_length_min_approx',
  "duration_seconds": "duration_seconds",
  'publishedAt': 'vid_upload_date',
  'liveBroadcastContent': 'vid_livestreamed',
  'viewCount':  'vid_views',
  'likeCount': 'vid_likes',
  "commentCount": 'vid_comments',
  "favoriteCount" :'vid_favorites',
  "captions_raw": 'caption_text_raw',
  "caption_text_flag": "caption_text_flag",
  "meeting_date": "meeting_date",
  "video_link": "video_link",
  "thumbail": "thumbnail",
  
    }

In [19]:
# getting meeting dates

renamed_cols_df = merged_df.rename(columns=column_rename_map)

In [22]:
renamed_cols_df[[c for c in column_rename_map.values() if c in renamed_cols_df.columns]].sample(2)

,channel_id,channel_title,vid_id,vid_title,vid_desc,vid_length_min_approx,duration_seconds,vid_upload_date,vid_livestreamed,vid_views,vid_likes,vid_comments,vid_favorites,caption_text_raw,caption_text_flag
65,UCphQA4DKYX14O32KDdf_mSQ,Town of Fort Fairfield,h9o1i-YhAiQ,"February 16, 2022 Regular Town Council Meeting","Regular Town Council Meeting - February 16, 2022.",50.0,3000.0,2022-02-17T18:48:59Z,none,119,1,0,0,uh mr well at first let's uh sorry about that ...,True
16,UCphQA4DKYX14O32KDdf_mSQ,Town of Fort Fairfield,4MsL6hX9Gps,15Nov23 Regular Town Council Meeting,,50.0,2971.0,2023-11-21T13:40:58Z,none,137,0,0,0,get just go just go halfway in between right l...,True


In [21]:
renamed_cols_df[[c for c in column_rename_map.values() if c in renamed_cols_df.columns]].to_csv("../outputs/sample-with-transcripts.csv", index=False)

In [ ]:
dftemp_final_video_channel_list.columns

In [1]:
dftemp_final_video_channel_list

NameError: name 'dftemp_final_video_channel_list' is not defined